# SQL agent

## Setup

### Imports

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from app.databases.external_db import get_db
from app.test_datasets import TestData
from app.prompts import PromptGenerator
from app.examples import FEW_SHOT_EXAMPLES

from app.agents import OllamaFunctionsSQLAgent

from langsmith import Client

from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings


from langchain_experimental.llms.ollama_functions import OllamaFunctions

### LangSmith

In [3]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [4]:
llama3_embeddings = OllamaEmbeddings(
    model="llama3:8b", 
    temperature=0
)

llama3__chat = ChatOllama(
    model="llama3:instruct", 
    temperature=0
)
llama3_q8__chat = ChatOllama(
    model="llama3:8b-instruct-q8_0", 
    temperature=0
)

llama3__with_functions = OllamaFunctions(
    model="llama3:instruct",
    format="json",
    temperature=0
)
llama3_q8__with_functions = OllamaFunctions(
    model="llama3:8b-instruct-q8_0",
    format="json",
    temperature=0
)

### Connect to DB with Readonly role

In [5]:
db = get_db()

#### Check connection

In [6]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman'), (6, 'Donald'), (7, 'Douglas'), (8, 'Dwight'), (9, 'Earl'), (10, 'Edgar'), (11, 'Edmund'), (12, 'Edwin'), (13, 'Elliot'), (14, 'Eric'), (15, 'Ernest'), (16, 'Ethan'), (17, 'Ezekiel'), (18, 'Felix'), (19, 'Franklin'), (20, 'Frederick'), (21, 'Gabriel'), (22, 'Joseph'), (23, 'Joshua'), (24, 'Julian'), (25, 'Alice'), (26, 'Bob'), (27, 'Charlie'), (28, 'David'), (29, 'Emily'), (30, 'Frank'), (31, 'George'), (32, 'Helen'), (33, 'Irene'), (34, 'Jack'), (35, 'Kate'), (36, 'Leo'), (37, 'Mary'), (38, 'Nancy'), (39, 'Oliver'), (40, 'Paul'), (41, 'Qiana'), (42, 'Robert'), (43, 'Samantha'), (44, 'Thomas'), (45, 'Victoria')]"

-----
## Testing an agent

In [ ]:
prompt_generator = PromptGenerator().set_example_selector(
    examples=FEW_SHOT_EXAMPLES,
    embedding_llm=llama3_embeddings,
)

agent = OllamaFunctionsSQLAgent(
    ollama_functions=llama3__with_functions,
    chat_llm=llama3__chat,
    prompt_generator=prompt_generator,
    db=db
)

res = agent.run_agent(TestData.QUESTIONS[0])

In [8]:
print(res.return_values["output"])

The first 5 names of passengers, sorted by name, are: Alice, Bob, Charlie, Christofer, and David.
